In [9]:
from os.path import abspath
from pyspark.sql import SparkSession

# set default location for [spark-warehouse]
warehouse_location = abspath('spark-warehouse')

In [10]:
spark = (
    SparkSession
    .builder
    .appName("Delta Lake")
    .config("spark.jars.packages", "org.apache.spark:spark-avro_2.12:3.1.1,io.delta:delta-core_2.12:1.0.0,org.apache.spark:spark-sql-kafka-0-10_2.12:3.1.1")
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension")
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")
    .config("spark.sql.debug.maxToStringFields","1000")
    .config("spark.kryoserializer.buffer.max","128m")
    .enableHiveSupport()
    .getOrCreate()
)
spark.sparkContext.setLogLevel("WARN")

In [11]:
conf = spark.sparkContext._jsc.hadoopConfiguration()
conf.set("fs.gs.project.id", "hering-datalake-prod")
conf.set("fs.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFileSystem")
conf.set("fs.AbstractFileSystem.gs.impl", "com.google.cloud.hadoop.fs.gcs.GoogleHadoopFS")

---

In [12]:
from pyspark.sql.avro.functions import from_avro, to_avro
import pyspark.sql.functions as f
from urllib.request import urlopen
import json
from pyspark.sql.types import StringType
from dateutil import parser
from datetime import datetime
from datetime import timedelta

In [5]:
kafka_bootstrap = 'edh-kafka-bootstrap.ingestion.svc:9092'
topics = 'eih.bnsvher311.PRCF.CFHSTEOP'

In [6]:
df = (
    spark
    .read
    .format('kafka')
    .option('kafka.bootstrap.servers', kafka_bootstrap)
    .option('subscribe', topics)
    .option('startingOffsets', 'earliest')
    .load()
)

In [7]:
df.count()

22/06/01 15:56:24 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-6b9eb2ee-2ef6-4759-971b-831fa292ff49-driver-0-1, groupId=spark-kafka-relation-6b9eb2ee-2ef6-4759-971b-831fa292ff49-driver-0] Error while fetching metadata with correlation id 2 : {eih.bnsvher311.PRCF.CFHSTEOP=LEADER_NOT_AVAILABLE}
22/06/01 15:56:24 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-6b9eb2ee-2ef6-4759-971b-831fa292ff49-driver-0-1, groupId=spark-kafka-relation-6b9eb2ee-2ef6-4759-971b-831fa292ff49-driver-0] Error while fetching metadata with correlation id 7 : {eih.bnsvher311.PRCF.CFHSTEOP=LEADER_NOT_AVAILABLE}
22/06/01 15:56:25 WARN NetworkClient: [Consumer clientId=consumer-spark-kafka-relation-6b9eb2ee-2ef6-4759-971b-831fa292ff49-driver-0-1, groupId=spark-kafka-relation-6b9eb2ee-2ef6-4759-971b-831fa292ff49-driver-0] Error while fetching metadata with correlation id 8 : {eih.bnsvher311.PRCF.CFHSTEOP=LEADER_NOT_AVAILABLE}
22/06/01 15:56:25 WARN NetworkClient: [Consume

0

In [8]:
stream_estoque.printSchema()

NameError: name 'stream_estoque' is not defined

In [ ]:
stream_estoque.show(1)

In [ ]:
diferenca = timedelta(hours = 5)

In [ ]:
data = datetime.today() - diferenca

In [ ]:
print(str(data))

In [ ]:
print(str(data)[11:13])

In [ ]:
dados = stream_estoque.filter(stream_estoque.timestamp >= data).count()

In [ ]:
stream_estoque.filter(stream_estoque.timestamp >= data).count()

In [ ]:
print(dados)

In [ ]:
# dados = stream_estoque.filter(stream_estoque.timestamp >= data).count()
# print(dados)
# Resultado  15637570

In [ ]:
teste = (
    dados
    .write
    .partitionBy('year', 'month', 'day', 'hour')
    .format('csv')
    .save('/work/sql')
)   

In [ ]:
query2 = (
    stream_venda
    .writeStream
    .partitionBy('year', 'month', 'day', 'hour')
    .format('avro')
    .option('checkpointLocation', landing_path_venda + 'checkpoint')
    .start(landing_path_venda)
)   